In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Alpha-beta pruning algorithm

### Dependencies

In [ ]:
!pip install python-chess
!pip install import_ipynb

     |████████████████████████████████| 147 kB 18.7 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=dbcf2e163e95ea06fec24b592188cfc5f992359ce2983bacbba9118e6646c17a
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import chess
import chess.gaviota
import random
import signal
import time
from typing import Union
from enum import Enum

import import_ipynb
import Util
from Globals import *

importing Jupyter notebook from Util.ipynb


importing Jupyter notebook from Globals.ipynb


#### chess.Board.get_best_move_alphabeta
Finds the best move to make based on the alpha-beta pruning algorithm. This algorithm works as follows:
- Iterate over all legal moves in the current position.
- For each move, find the best possible score after making this move.
    - This is done by calling the alpha-beta pruning function recursively: increasing the current iteration by 1 and switching turns.
    - If a move yields a worse score than the previously examined move in the hierarchy, it cannot be the optimal move. Therefore, such moves are not evaluated further.
- Find the maximum (if it's the AI's turn) or the minimum (if it's other player's/AI's turn) score of all legal moves, alongside the move that was able to reach this optimized state. This is the move that the algorithm recommends.

Optionally, the algorithm uses memoization, which is a type of caching. This works by mapping a board state, plus iteration details, to the corresponding score and move and storing it in a cache dictionary. If, during a later iteration, the same board state is reached on the same iteration, these values can be read from the cache.

__This function is implemented recursively.__

###### <b><u>Arguments</u></b>
__cache__ _(dict)_ : A cache dictionary if memoization is desired, or None if memoization should be disabled.  
__alpha__ _(int)_ : The "alpha" value in the current iteration. On the first iteration, this value should be strongly negative.  
__beta__ _(int)_ : The "beta" value in the current iteration. On the first iteration, this value should be strongly positive.  
__ai_turn__ _(bool)_ : Whether or not the current turn is of the AI that started the search. If this is the case, the score should be maximized. Otherwise, the score should be minimized.  
__color__ _(chess.Color)_ : The color of the player whose turn it currently is.  
__endgame_tablebase__ _(Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase])_ : The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  
__iteration__ _(int)_ : The depth of the search (amount of moves currently looking ahead).  
__max_iterations__ _(int)_ : The maximum depth of the search.  
__using_progressive_deepening__ _(bool)_ : Whether or not the search is part of a progressive deepening search.  
__last_eval_score__ _(int)_ : The score provided by the previous evaluation in the search.  

###### <b><u>Returns <i>(int, chess.Move, bool)</i></u></b>
- The board score after making the recommended best move.
- The recommended best move to make.
- Whether or not the endgame library was used to find the move.
- Whether or not the endgame library was used to find the move.

###### <b><u>Side effects</u></b>
- If a cache dictionary is provided, new values may be added to this dictionary.
- If the search is interrupted, the board may be in a different state than when the search started.

In [ ]:
def get_best_move_alphabeta(
    self,
    cache: dict,
    alpha: int,
    beta: int,
    ai_turn: bool,
    color: chess.Color,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase],
    iteration: int,
    max_iterations: int,
    using_progressive_deepening: bool,
    last_eval_score: int
) -> (int, chess.Move, bool):

    state = self.get_state_string()
    if cache is not None:                
        alpha, beta, cached_entry = self.get_alphabeta_cache(cache, iteration, max_iterations, state, alpha, beta)
        if cached_entry is not None: return cached_entry

    original_color = color if ai_turn else not color
    result_score = self.get_search_result_if_finished(original_color, iteration, max_iterations, last_eval_score)
    if result_score is not None:
        return result_score, None, False

    # If the game has not finished, check additional moves using the alpha-beta pruning algorithm
    best_score = Globals.MAX_EVALUATION_SCORE * (-1 if ai_turn else 1)
    best_move = None
    best_move_used_endgame = False

    # Single value extension
    if len(self.legal_moves) == 1:
        return self.get_best_move_alphabeta(cache, alpha, beta, not ai_turn, not color, endgame_tablebase, iteration,
                                            max_iterations, using_progressive_deepening, last_eval_score)

    available_moves = self.get_sorted_moves(cache, ai_turn, max_iterations) if using_progressive_deepening else self.legal_moves
    for move in available_moves:
        eval_score, used_endgame_anywhere = self.evaluate_move(color, not ai_turn, last_eval_score, iteration, move, endgame_tablebase)
        self.push(move)

        if self.is_game_over():
            score_after_move = eval_score
        else:
            score_after_move, _, used_endgame = self.get_best_move_alphabeta(cache, alpha, beta, not ai_turn, not color, endgame_tablebase,
                                                                             iteration + 1, max_iterations, using_progressive_deepening,
                                                                             eval_score)
            used_endgame_anywhere = used_endgame_anywhere or used_endgame

        self.pop()
                                                               
        if (ai_turn and score_after_move > best_score) or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
            best_move_used_endgame = used_endgame_anywhere

        if ai_turn:
            if best_score >= beta:
                return best_score, best_move, best_move_used_endgame
            if best_score > alpha:
                alpha = best_score
        else:
            if best_score <= alpha:
                return best_score, best_move, best_move_used_endgame
            if best_score < beta:
                beta = best_score

    result = (best_score, best_move, best_move_used_endgame)
    if cache is not None:
        self.store_alphabeta_cache(cache, iteration, max_iterations, state, alpha, beta, result)
        
    return result

chess.Board.get_best_move_alphabeta = get_best_move_alphabeta
del get_best_move_alphabeta

In [ ]:
class Comparisons(Enum):
    EQUAL = 0
    LTE = 1 # Less than or equal to
    GTE = 2 # Greater than or equal to

In [ ]:
def get_alphabeta_cache(
    self,
    cache: dict,
    iteration: int,
    max_iterations: int,
    state: str,
    alpha: int,
    beta: int
) -> (int, int, (int, chess.Move, bool)):

    # Note: if a result is returned (i.e. not None) then alpha and beta don't matter anymore

    key = (iteration, max_iterations, state)
    if key not in cache:
        return alpha, beta, None

    flag, result = cache[key]
    score, _, _ = result
    if flag == Comparisons.EQUAL:
        return alpha, beta, result
    if flag == Comparisons.LTE:
        if score <= alpha:
            return alpha, beta, result
        if score < beta: # alpha < score < beta
            return alpha, score, None
    if flag == Comparisons.GTE:
        if beta <= score:
            return alpha, beta, result
        if alpha < score: # alpha < score < beta
            return score, beta, None

    # Invalid flag or no optimization possible
    return alpha, beta, None

chess.Board.get_alphabeta_cache = get_alphabeta_cache
del get_alphabeta_cache

In [ ]:
def store_alphabeta_cache(
    self,
    cache: dict,
    iteration: int,
    max_iterations: int,
    state: str,
    alpha: int,
    beta: int,
    result: tuple
) -> None:

    key = (iteration, max_iterations, state)
    score, _, _ = result

    if score <= alpha:
        cache[key] = (Comparisons.LTE, result)
    elif score < beta: # alpha < score < beta
        cache[key] = (Comparisons.EQUAL, result)
    else: # beta <= score
        cache[key] = (Comparisons.GTE, result)

chess.Board.store_alphabeta_cache = store_alphabeta_cache
del store_alphabeta_cache

In [ ]:
def alpha_beta_value_cache(self, cache: dict, move: chess.Move, max_iterations: int) -> int:
    if max_iterations < 1:
        return 0
        
    self.push(move)
    state = self.get_state_string()
    self.pop()
    
    _, result = cache.get((0, max_iterations, state), ('=', (0, None, False)))
    score, _, _ = result
    return score

chess.Board.alpha_beta_value_cache = alpha_beta_value_cache
del alpha_beta_value_cache

In [ ]:
def get_sorted_moves(self, cache: dict, ai_turn: bool, max_iterations: int) -> list:
    return sorted(self.legal_moves, reverse=ai_turn, key=lambda move: self.alpha_beta_value_cache(cache, move, max_iterations - 2))

chess.Board.get_sorted_moves = get_sorted_moves
del get_sorted_moves

#### make_move_alphabeta
Finds the best possible move according to the alpha-beta pruning algorithm and pushes it onto the move stack.

###### <b><u>Arguments</u></b>
__board__ _(chess.Board)_ : The board to push the move to.  
__color__ _(chess.Color)_ : The color of the player that makes the move.  
__search_depth__ _(int)_ : The iteration depth of the alpha-beta search.  
__endgame_tablebase__ _(Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase])_ : The endgame tablebase attached to the game, which serves as a shortcut for ideal moves in the endgame.  

###### <b><u>Returns <i>(int, chess.Move, bool, bool)</i></u></b>
- The evaluated score of the best possible move.
- The best possible move that was found.
- Whether or not the endgame library was used to find the move.
- Whether or not the endgame library was used to find the move.

###### <b><u>Side effects</u></b>
- The best possible move is pushed to the move stack of the board.

In [ ]:
def make_move_alphabeta(
    board: chess.Board,
    color: chess.Color,
    search_depth: int,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]
) -> (int, chess.Move, bool):

    score, move, used_endgame = board.get_best_move_alphabeta(
        cache={}, 
        alpha=-Globals.MAX_EVALUATION_SCORE, 
        beta=Globals.MAX_EVALUATION_SCORE, 
        ai_turn=True, 
        color=color,
        endgame_tablebase=endgame_tablebase,
        iteration=0, 
        max_iterations=search_depth,
        using_progressive_deepening=False,
        last_eval_score=0 # The starting board score doesn't matter, it's evaluated by score difference
    )
    
    board.push(move)
    return score, move, used_endgame

In [ ]:
def make_move_alphabeta_progressive_deepening(
    board: chess.Board,
    color: chess.Color,
    search_depth: int,
    endgame_tablebase: Union[chess.gaviota.NativeTablebase, chess.gaviota.PythonTablebase]
) -> (int, chess.Move, bool):

    cache = {}
    score, move, used_endgame = 0, None, False

    for limit in range(1, search_depth + 1):
        score, move, used_endgame = board.get_best_move_alphabeta(
            cache=cache,
            alpha=-Globals.MAX_EVALUATION_SCORE,
            beta=Globals.MAX_EVALUATION_SCORE,
            ai_turn=True,
            color=color,
            endgame_tablebase=endgame_tablebase,
            iteration=0,
            max_iterations=limit,
            using_progressive_deepening=True,
            last_eval_score=0 # The starting board score doesn't matter, it's evaluated by score difference
        )

        # If a checkmate in favor of the AI has been found, there's no reason to search any further
        if score == Globals.EVALUATION_SCORE_CHECKMATE:
            return score, move, used_endgame

    board.push(move)
    return score, move, used_endgame

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>